In [1]:
# %cd /home/ubu-admin/Developer/tensorflow-wavelets
%cd /workspaces/OpenDVCW
import os
import load
import OpenDVCW
import tensorflow as tf
from math import log10, sqrt
import cv2
import numpy as np

/workspaces/OpenDVCW


In [2]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [3]:
BATCH_SIZE = 1
Height = 240
Width = 240
lmbda = 2048
print("Loading model")
model = tf.keras.models.load_model('model_save_wavelets_L__2048_5_240x240')
# Show the model architecture
model.summary()

Loading model


2022-04-03 16:54:55.631618: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 16:54:55.668469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 16:54:55.669334: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 16:54:55.674860: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "open_dvc"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mv_analysis (Sequential)    (None, None, None, 128)   659456    
                                                                 
 mv_synthesis (Sequential)   (None, None, None, 2)     642824    
                                                                 
 res_analysis (Sequential)   (None, None, None, 128)   1552640   
                                                                 
 res_synthesis (Sequential)  (None, None, None, 3)     1536015   
                                                                 
 wavelets_optical_flow (Wave  multiple                 240050    
 letsOpticalFlow)                                                
                                                                 
 motion_compensation (Motion  multiple                 486467    
 Compensation)                                            

2022-04-03 16:55:50.349965: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [4]:
i=0
path = load.load_path_n("folder_cloud_test.npy", 0)
out_bin = "Test_com/test{}.bin".format(i)
out_decom = "Test_com/testdcom{}.png".format(i)
p_on_test = "Test_com/test_p_frame{}.png".format(i)
i_on_test = "Test_com/test_i_frame{}.png".format(i)

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)

OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))

OpenDVCW.compress(model, i_frame, p_frame, out_bin, 240, 240)



OpenDVCW.decompress(model, i_frame, out_bin, out_decom, 240, 240)

original = cv2.imread(p_on_test)
compressed = cv2.imread(out_decom)
bin_size = os.path.getsize(out_bin)
value = PSNR(original, compressed)

print("bin size: ", bin_size , "psnr: ", value)

/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00049/0289/im1.png
compress


2022-04-03 16:56:15.496812: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


FileNotFoundError: [Errno 2] No such file or directory: 'out_bin'